## test

In [ ]:
import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

## import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, Flatten
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model, Model
from livelossplot.inputs.tf_keras import PlotLossesCallback
from tensorflow import keras

## lable preparation

In [ ]:
tr_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/train_kp.csv')
vl_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/val_kp.csv')
ts_df = pd.read_csv('F:/poorya/datasets/ThermalFaceDatabase/test_kp.csv')

In [ ]:
def df_to_ary(df):
    ary = np.zeros((len(df), 136), dtype = int)
    for i in range(len(df)):
        rep_x = [j for j in range(len(df['x'][i])) if df['x'][i].startswith(',', j)]
        rep_y = [j for j in range(len(df['y'][i])) if df['y'][i].startswith(',', j)]
        ary[i, 0] = int(df['x'][i][1:rep_x[0]]) / 2
        ary[i, 67] = int(df['x'][i][rep_x[-1] + 1:-1]) / 2
        ary[i, 68] = (int(df['y'][i][1:rep_y[0]]) - 128) / 2
        ary[i, 135] = (int(df['y'][i][rep_y[-1] + 1:-1]) - 128) / 2
        for k in range(66):
            ary[i, k + 1] = int(df['x'][i][rep_x[k] + 1:rep_x[k + 1]]) / 2
            ary[i, k + 69] = (int(df['y'][i][rep_y[k] + 1:rep_y[k + 1]]) - 128) / 2

    return ary


tr_ary = df_to_ary(tr_df)
ts_ary = df_to_ary(ts_df)
vl_ary = df_to_ary(vl_df)

## functions

In [ ]:
def window_partition(x, window_size):
    _, height, width, channels = x.shape
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        x, shape = (-1, patch_num_y, window_size, patch_num_x, window_size, channels)
        )
    x = tf.transpose(x, (0, 1, 3, 2, 4, 5))
    windows = tf.reshape(x, shape = (-1, window_size, window_size, channels))
    return windows


def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        windows,
        shape = (-1, patch_num_y, patch_num_x, window_size, window_size, channels),
        )
    x = tf.transpose(x, perm = (0, 1, 3, 2, 4, 5))
    x = tf.reshape(x, shape = (-1, height, width, channels))
    return x


class DropPath(layers.Layer):
    def __init__(self, drop_prob=None, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, x):
        input_shape = tf.shape(x)
        batch_size = input_shape[0]
        rank = x.shape.rank
        shape = (batch_size,) + (1,) * (rank - 1)
        random_tensor = (1 - self.drop_prob) + tf.random.uniform(shape, dtype = x.dtype)
        path_mask = tf.floor(random_tensor)
        output = tf.math.divide(x, 1 - self.drop_prob) * path_mask
        return output

In [ ]:
class WindowAttention(layers.Layer):
    def __init__(
        self, dim, window_size, num_heads, qkv_bias=True, dropout_rate=0.0, **kwargs
        ):
        super().__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = layers.Dense(dim * 3, use_bias = qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

    def build(self, input_shape):
        num_window_elements = (2 * self.window_size[0] - 1) * (
                               2 * self.window_size[1] - 1
        )
        self.relative_position_bias_table = self.add_weight(
            name = 'W',
            shape = (num_window_elements, self.num_heads),
            initializer = tf.initializers.Zeros(),
            trainable = True,
            )
        coords_h = np.arange(self.window_size[0])
        coords_w = np.arange(self.window_size[1])
        coords_matrix = np.meshgrid(coords_h, coords_w, indexing = "ij")
        coords = np.stack(coords_matrix)
        coords_flatten = coords.reshape(2, -1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)

        self.relative_position_index = tf.Variable(
            initial_value = tf.convert_to_tensor(relative_position_index), trainable = False
            )

    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = tf.reshape(x_qkv, shape = (-1, size, 3, self.num_heads, head_dim))
        x_qkv = tf.transpose(x_qkv, perm = (2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = tf.transpose(k, perm = (0, 1, 3, 2))
        attn = q @ k

        num_window_elements = self.window_size[0] * self.window_size[1]
        relative_position_index_flat = tf.reshape(
            self.relative_position_index, shape = (-1,)
            )
        relative_position_bias = tf.gather(
            self.relative_position_bias_table, relative_position_index_flat
            )
        relative_position_bias = tf.reshape(
            relative_position_bias, shape = (num_window_elements, num_window_elements, -1)
            )
        relative_position_bias = tf.transpose(relative_position_bias, perm = (2, 0, 1))
        attn = attn + tf.expand_dims(relative_position_bias, axis = 0)

        if mask is not None:
            nW = mask.get_shape()[0]
            mask_float = tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis = 1), axis = 0), tf.float32
                )
            attn = (
                                   tf.reshape(attn, shape = (-1, nW, self.num_heads, size, size))
                                   + mask_float
            )
            attn = tf.reshape(attn, shape = (-1, self.num_heads, size, size))
            attn = keras.activations.softmax(attn, axis = -1)
        else:
            attn = keras.activations.softmax(attn, axis = -1)
        attn = self.dropout(attn)

        x_qkv = attn @ v
        x_qkv = tf.transpose(x_qkv, perm = (0, 2, 1, 3))
        x_qkv = tf.reshape(x_qkv, shape = (-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv

In [ ]:
class SwinTransformer(layers.Layer):
    def __init__(
        self,
        dim,
        num_patch,
        num_heads,
        window_size=7,
        shift_size=0,
        num_mlp=1024,
        qkv_bias=True,
        dropout_rate=0.0,
        **kwargs,
        ):
        super().__init__(**kwargs)

        self.dim = dim  # number of input dimensions
        self.num_patch = num_patch  # number of embedded patches
        self.num_heads = num_heads  # number of attention heads
        self.window_size = window_size  # size of window
        self.shift_size = shift_size  # size of window shift
        self.num_mlp = num_mlp  # number of MLP nodes

        self.norm1 = layers.LayerNormalization(epsilon = 1e-5)
        self.attn = WindowAttention(
            dim,
            window_size = (self.window_size, self.window_size),
            num_heads = num_heads,
            qkv_bias = qkv_bias,
            dropout_rate = dropout_rate,
            )
        self.drop_path = DropPath(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon = 1e-5)

        self.mlp = keras.Sequential(
            [
                layers.Dense(num_mlp),
                layers.Activation(keras.activations.gelu),
                layers.Dropout(dropout_rate),
                layers.Dense(dim),
                layers.Dropout(dropout_rate),
                ]
            )

        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size == 0:
            self.attn_mask = None
        else:
            height, width = self.num_patch
            h_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
                )
            w_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
                )
            mask_array = np.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_array = tf.convert_to_tensor(mask_array)

            # mask array to windows
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = tf.reshape(
                mask_windows, shape = [-1, self.window_size * self.window_size]
                )
            attn_mask = tf.expand_dims(mask_windows, axis = 1) - tf.expand_dims(
                mask_windows, axis = 2
                )
            attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
            attn_mask = tf.where(attn_mask == 0, 0.0, attn_mask)
            self.attn_mask = tf.Variable(initial_value = attn_mask, trainable = False)

    def call(self, x):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = tf.reshape(x, shape = (-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = tf.roll(
                x, shift = [-self.shift_size, -self.shift_size], axis = [1, 2]
                )
        else:
            shifted_x = x

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = tf.reshape(
            x_windows, shape = (-1, self.window_size * self.window_size, channels)
            )
        attn_windows = self.attn(x_windows, mask = self.attn_mask)

        attn_windows = tf.reshape(
            attn_windows, shape = (-1, self.window_size, self.window_size, channels)
            )
        shifted_x = window_reverse(
            attn_windows, self.window_size, height, width, channels
            )
        if self.shift_size > 0:
            x = tf.roll(
                shifted_x, shift = [self.shift_size, self.shift_size], axis = [1, 2]
                )
        else:
            x = shifted_x

        x = tf.reshape(x, shape = (-1, height * width, channels))
        x = self.drop_path(x)
        x = x_skip + x
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = x_skip + x
        return x

In [ ]:
class PatchExtract(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size_x = patch_size[0]
        self.patch_size_y = patch_size[0]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = (1, self.patch_size_x, self.patch_size_y, 1),
            strides = (1, self.patch_size_x, self.patch_size_y, 1),
            rates = (1, 1, 1, 1),
            padding = "VALID",
            )
        patch_dim = patches.shape[-1]
        patch_num = patches.shape[1]
        return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))


class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patch = num_patch
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim = num_patch, output_dim = embed_dim)

    def call(self, patch):
        pos = tf.range(start = 0, limit = self.num_patch, delta = 1)
        return self.proj(patch) + self.pos_embed(pos)


class PatchMerging(tf.keras.layers.Layer):
    def __init__(self, num_patch, embed_dim):
        super().__init__()
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias = False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.get_shape().as_list()
        x = tf.reshape(x, shape = (-1, height, width, C))
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = tf.concat((x0, x1, x2, x3), axis = -1)
        x = tf.reshape(x, shape = (-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, ary, batch_size):
        self.n = df['images'].tolist()
        self.df = df
        self.ary = ary
        self.batch_size = batch_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.n) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp = [self.n[k] for k in indexes]
        x, y = self.__data_generation(list_IDs_temp, indexes)
        return x, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.n))

    def __data_generation(self, list_IDs_temp, indexes):
        images = np.empty((self.batch_size, 384, 384, 1))
        kp = np.empty((self.batch_size, 136))

        for i, ID in enumerate(list_IDs_temp):
            train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ID, color_mode = "grayscale", target_size = (
            384, 512)))
            train_img = train_img[:, 64:448]
            train_img /= 255.0
            images[i,] = train_img

            kp[i,] = self.ary[indexes[i], :]

        return images, kp


bs = 2
train_gen = DataGenerator(tr_df, tr_ary, bs)
val_gen = DataGenerator(vl_df, vl_ary, bs)
test_gen = DataGenerator(ts_df, ts_ary, bs)

## model structure

In [ ]:
def PSNR(super_resolution, high_resolution):
    psnr_value = tf.image.psnr(high_resolution, super_resolution, max_val = 255)[0]
    return psnr_value


def SSIMLoss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))


def vgg_block(layer_in, n_filters, n_conv):
    for _ in range(n_conv):
        layer_in = Conv2D(n_filters, (3, 3), padding = 'same', activation = 'relu')(layer_in)
    layer_in = BatchNormalization()(layer_in)
    layer_in = AveragePooling2D((2, 2), strides = (2, 2))(layer_in)
    return layer_in


def get_model():

    ######################################  INITIALIZATION  ########################################

    patch_size = (2, 2)
    num_heads = 8
    embed_dim = 64
    qkv_bias = True
    window_size = 2
    shift_size = 1
    image_dimension = 384
    num_mlp = 512
    dropout_rate = 0.1

    num_patch_x = 384 // patch_size[0]
    num_patch_y = 384 // patch_size[1]

    weight_decay = 0.0001
    label_smoothing = 0.05

    drp = 0.0
    act = 'relu'

    ########################################  BACKBONE  ############################################

    img_input = Input(shape = (384, 384, 1))

    layer = vgg_block(img_input, 8, 2)
    layer = vgg_block(layer, 16, 2)
    layer = vgg_block(layer, 32, 2)
    layer = vgg_block(layer, 64, 2)
    layer = vgg_block(layer, 128, 2)
    layer = vgg_block(layer, 256, 2)

    x1 = Flatten()(layer)

    #########################################  SWIN T  #############################################

    x = layers.experimental.preprocessing.RandomCrop(image_dimension, image_dimension)(img_input)
    x = layers.experimental.preprocessing.RandomFlip("horizontal")(x)
    x = PatchExtract(patch_size)(x)
    x = PatchEmbedding(num_patch_x * num_patch_y, embed_dim)(x)
    x = SwinTransformer(
        dim = embed_dim,
        num_patch = (num_patch_x, num_patch_y),
        num_heads = num_heads,
        window_size = window_size,
        shift_size = 0,
        num_mlp = num_mlp,
        qkv_bias = qkv_bias,
        dropout_rate = dropout_rate)(x)
    x = SwinTransformer(
        dim = embed_dim,
        num_patch = (num_patch_x, num_patch_y),
        num_heads = num_heads,
        window_size = window_size,
        shift_size = shift_size,
        num_mlp = num_mlp,
        qkv_bias = qkv_bias,
        dropout_rate = dropout_rate)(x)
    x = PatchMerging((num_patch_x, num_patch_y), embed_dim = embed_dim)(x)
    x2 = layers.GlobalAveragePooling1D()(x)

    ########################################  FLATTEN  ############################################

    x = layers.Concatenate()([x1, x2])

    x = Dense(1024, activation = act)(x)
    x = Dropout(drp)(x)
    x = Dense(512, activation = act)(x)
    x = Dropout(drp)(x)

    last = Dense(136)(x)

    return Model(inputs = img_input, outputs = last)


metrics = [tf.keras.metrics.MeanAbsolutePercentageError()]
opt = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
loss = tf.keras.losses.MeanSquaredError()

model = get_model()
model.compile(loss = loss, optimizer = opt, metrics = metrics)
model.summary()

## train

In [ ]:
def tr():
    chk = ModelCheckpoint(filepath = 'supervised_checkpoint', monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
    ers = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 0)
    rduce_lr = ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 5, verbose = 1, mode = "min", min_lr = 0.000001)
    vk = PlotLossesCallback()
    lgg = tf.keras.callbacks.CSVLogger('epoches.csv')
    return [vk, lgg, chk, rduce_lr]


cll = tr()
history = model.fit(train_gen,
                    validation_data = val_gen,
                    batch_size = bs,
                    epochs = 200,
                    verbose = 1,
                    callbacks = cll,
                    steps_per_epoch = np.shape(tr_ary)[0] // bs,
                    validation_steps = np.shape(vl_ary)[0] // bs
                    )

## evaluation

In [ ]:
img = np.zeros((1, 384, 384, 1)).astype('double')
myi = 193

train_igg = image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + ts_df['images'][
    myi], color_mode = "grayscale", target_size = (384, 512))
train_img = image.img_to_array(train_igg)
train_img /= 255.0
img[0, :, :, :] = train_img[:, 64:448]
img = np.array(img)

kp = ts_ary[myi]

classifier = load_model('supervised_checkpoint')
lbl_c = classifier.predict(img)

# summ = 0
# for i in range(100):
#     start = time()
#     lbl_c = classifier.predict(img)
#     end   = time()
#     summ  = summ + (end - start)

x = lbl_c[0, 0:68]
y = lbl_c[0, 68:]

xx = kp[0:68]
yy = kp[68:]

plt.figure(figsize = (20, 10))
plt.imshow(img[0], cmap = 'gray')
plt.scatter(y, x)
plt.scatter(yy, xx)
plt.show()

In [ ]:
n = ts_df['images'].tolist()
x = np.zeros((len(n), 384, 384, 1))

for i in range(len(n)):
    train_img = image.img_to_array(image.load_img('F:/poorya/datasets/ThermalFaceDatabase/' + n[
        i], color_mode = "grayscale", target_size = (384, 512)))
    train_img /= 255.0
    x[i] = train_img[:, 64:448]

pred = classifier.predict(x)

summ = 0
one_by_one = []
for i in range(np.shape(pred)[0]):
    y_pred = pred[i]
    y_true = ts_ary[i]
    x1 = y_pred[42:48].mean()
    x2 = y_pred[36:42].mean()
    y1 = y_pred[42 + 68:48 + 68].mean()
    y2 = y_pred[36 + 68:42 + 68].mean()
    summ += np.linalg.norm(y_true - y_pred) / (math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) * 68)
    one_by_one.append(np.linalg.norm(y_true - y_pred) / (math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) * 68))

print(100 * summ / np.shape(pred)[0])

In [ ]:
N = 100

# initializing random values 
data = np.random.randn(N)

# getting data of the histogram 
count, bins_count = np.histogram(one_by_one, bins = 100)

# finding the PDF of the histogram using count values 
pdf = count / sum(count)

# using numpy np.cumsum to calculate the CDF 
# We can also find using the PDF values by looping and adding 
cdf = np.cumsum(pdf)

to_save = np.zeros((2, len(cdf)))
to_save[0] = bins_count[1:]
to_save[1] = cdf

#np.save('C:/Users/DrBah/Desktop/keypoint images/03 - Results/Keypoint detection/05 NME for different pretexts - bar plot/Gender_inpainting_subject_rotation.npy', to_save)

# plotting PDF and CDF 
plt.plot(to_save[0], to_save[1], label = "CDF")
plt.legend()